# 보이지 않는 손

## 일치율

### 또래지명에 의한 척도별 점수 산출

In [2]:
import pandas as pd
import numpy as np

In [4]:
student_df = pd.read_excel('5학년_1반_전수(AC000120255019FA1)/AC000120255019FA1_학생 명단.xlsx')
student_list = student_df['학생 이름'].to_list()

point_df = pd.read_excel('5학년_1반_전수(AC000120255019FA1)/AC000120255019FA1_또래지명_응답.xlsx')

student_evaluation_df = pd.read_excel('5학년_1반_전수(AC000120255019FA1)/AC000120255019FA1_교사_학생평가_응답.xlsx')

# 지명수 구하는 함수
def count_point(df, student_list, item_no) :
    temp_df = df[df['문항번호'].isin(item_no)]
    count_list = []
    for student in student_list :
        if len(temp_df[temp_df['지명받은 학생']==student]) >= 1 :
            freq = len(temp_df[temp_df['지명받은 학생']==student])
        elif len(temp_df[temp_df['지명받은 학생']==student]) == 0 :
            freq = 0
        count_list.append(freq)
    count_point_df = pd.DataFrame({'학생명': student_list, '지명수': count_list})
    return count_point_df
    

In [5]:
# 학생 명단과 또래지명을 통해서 척도별 점수 산출 함수
def calculate_scale_scores_by_peer_nomination(student_list, point_df):
    """
    학생 명단과 또래지명 데이터, 문항 번호 리스트를 받아
    학생별로 척도별 점수를 반환하는 함수
    """
    # 정규화
    def normalize(z):
        if z >= 0.84:
            return 5
        elif z >= 0.25:
            return 4
        elif z >= -0.25:
            return 3
        elif z >= -0.84:
            return 2
        else:
            return 1
    
    # 인기
    popularity_item_no = [1]
    popularity_count_df = count_point(point_df, student_list, popularity_item_no)
    popularity_count_mean = popularity_count_df['지명수'].mean()

    popularity_count_df['인기Z점수'] = (popularity_count_df['지명수'] - popularity_count_mean) / popularity_count_df['지명수'].std(ddof=0) # 인기 지명수 Z점수
    popularity_count_df['인기_정규화'] = popularity_count_df['인기Z점수'].apply(normalize)
    # 사회적 선호
    social_preference_item_no = [2]
    social_preference_count_df = count_point(point_df, student_list, social_preference_item_no) # 사회적 선호 지명수
    social_preference_count_mean = social_preference_count_df['지명수'].mean() # 사회적 선호 지명수 학급평균
    social_preference_count_df['사회적선호Z점수'] = (social_preference_count_df['지명수'] - social_preference_count_mean) / social_preference_count_df['지명수'].std(ddof=0) # 사회적 선호 지명수 Z점수
    social_preference_count_df['사회적선호_정규화'] = social_preference_count_df['사회적선호Z점수'].apply(normalize)
    # 외현적 공격성
    overt_aggression_item_no = [4,5]
    overt_aggression_count_df = count_point(point_df, student_list, overt_aggression_item_no) # 외현적 공격성 지명수
    overt_aggression_count_mean = overt_aggression_count_df['지명수'].mean() # 외현적 공격성 지명수 학급평균
    overt_aggression_count_df['외현적공격성Z점수'] = (overt_aggression_count_df['지명수'] - overt_aggression_count_mean) / overt_aggression_count_df['지명수'].std(ddof=0) # 외현적 공격성 지명수 Z점수
    overt_aggression_count_df['외현적공격성_정규화'] = overt_aggression_count_df['외현적공격성Z점수'].apply(normalize)
    # 관계적 공격성
    relational_aggression_item_no = [6,7]
    relational_aggression_count_df = count_point(point_df, student_list, relational_aggression_item_no) # 관계적 공격성 지명수
    relational_aggression_count_mean = relational_aggression_count_df['지명수'].mean() # 관계적 공격성 지명수 학급평균
    relational_aggression_count_df['관계적공격성Z점수'] = (relational_aggression_count_df['지명수'] - relational_aggression_count_mean) / relational_aggression_count_df['지명수'].std(ddof=0) # 관계적 공격성 지명수 Z점수
    relational_aggression_count_df['관계적공격성_정규화'] = relational_aggression_count_df['관계적공격성Z점수'].apply(normalize)
    # 친사회성
    prosocial_item_no = [8,9]
    prosocial_count_df = count_point(point_df, student_list, prosocial_item_no) # 친사회성 지명수
    prosocial_count_mean = prosocial_count_df['지명수'].mean() # 친사회성 지명수 학급평균
    prosocial_count_df['친사회성Z점수'] = (prosocial_count_df['지명수'] - prosocial_count_mean) / prosocial_count_df['지명수'].std(ddof=0) # 친사회성 지명수 Z점수
    prosocial_count_df['친사회성_정규화'] = prosocial_count_df['친사회성Z점수'].apply(normalize)
    # 가해
    perpetrator_item_no = [10, 11]
    perpetrator_count_df = count_point(point_df, student_list, perpetrator_item_no)  # 가해 지명수
    perpetrator_count_mean = perpetrator_count_df['지명수'].mean()  # 가해 지명수 학급평균
    perpetrator_count_df['가해Z점수'] = (perpetrator_count_df['지명수'] - perpetrator_count_mean) / perpetrator_count_df['지명수'].std(ddof=0)  # 가해 지명수 Z점수
    perpetrator_count_df['가해_정규화'] = perpetrator_count_df['가해Z점수'].apply(normalize)
    # 피해
    victim_item_no = [12, 13]
    victim_count_df = count_point(point_df, student_list, victim_item_no)  # 피해 지명수
    victim_count_mean = victim_count_df['지명수'].mean()  # 피해 지명수 학급평균
    victim_count_df['피해Z점수'] = (victim_count_df['지명수'] - victim_count_mean) / victim_count_df['지명수'].std(ddof=0)  # 피해 지명수 Z점수
    victim_count_df['피해_정규화'] = victim_count_df['피해Z점수'].apply(normalize)
    # 교사 선호
    teacher_preference_item_no = [18]
    teacher_preference_count_df = count_point(point_df, student_list, teacher_preference_item_no)
    teacher_preference_count_mean = teacher_preference_count_df['지명수'].mean()
    teacher_preference_count_df['교사선호Z점수'] = (teacher_preference_count_df['지명수'] - teacher_preference_count_mean) / teacher_preference_count_df['지명수'].std(ddof=0)  # 교사 선호 지명수 Z점수
    teacher_preference_count_df['교사선호_정규화'] = teacher_preference_count_df['교사선호Z점수'].apply(normalize)
    # 교사 비선호
    teacher_non_preference_item_no = [19]
    teacher_non_preference_count_df = count_point(point_df, student_list, teacher_non_preference_item_no)
    teacher_non_preference_count_mean = teacher_non_preference_count_df['지명수'].mean()
    teacher_non_preference_count_df['교사비선호Z점수'] = (teacher_non_preference_count_df['지명수'] - teacher_non_preference_count_mean) / teacher_non_preference_count_df['지명수'].std(ddof=0)  # 교사 비선호 지명수 Z점수
    teacher_non_preference_count_df['교사비선호_정규화'] = teacher_non_preference_count_df['교사비선호Z점수'].apply(normalize)

    return pd.DataFrame({
        '학생명': student_list,
        '인기_정규화': popularity_count_df['인기_정규화'],
        '사회적선호_정규화': social_preference_count_df['사회적선호_정규화'],
        '외현적공격성_정규화': overt_aggression_count_df['외현적공격성_정규화'],
        '관계적공격성_정규화': relational_aggression_count_df['관계적공격성_정규화'],
        '친사회성_정규화': prosocial_count_df['친사회성_정규화'],
        '가해_정규화': perpetrator_count_df['가해_정규화'],
        '피해_정규화': victim_count_df['피해_정규화'],
        '교사선호_정규화': teacher_preference_count_df['교사선호_정규화'],
        '교사비선호_정규화': teacher_non_preference_count_df['교사비선호_정규화']
    })

In [6]:
calculate_scale_scores_by_peer_nomination_df= calculate_scale_scores_by_peer_nomination(student_list, point_df)
print(calculate_scale_scores_by_peer_nomination_df)

    학생명  인기_정규화  사회적선호_정규화  외현적공격성_정규화  관계적공격성_정규화  친사회성_정규화  가해_정규화  피해_정규화  \
0   김하윤       4          1           4           5         2       1       2   
1   이준서       2          3           1           3         3       1       1   
2   박서연       3          3           5           2         5       1       1   
3   최민준       4          1           1           1         3       4       2   
4   정예은       5          3           5           2         2       3       2   
5   강도윤       1          5           4           5         2       1       1   
6   오서준       1          5           5           1         3       3       4   
7   윤지우       3          3           2           4         3       1       5   
8   한예진       2          1           4           1         3       2       5   
9   장하람       3          1           1           4         1       4       2   
10  조하린       1          1           2           4         2       4       4   
11  배수아       2          4           3  

In [7]:
'''
다 맞음
'''

'\n다 맞음\n'

### 학생평가에 의한 척도별 점수 산출

In [8]:
student_evaluation_df

,학생 이름,문항번호,응답값
0,김하윤,1,5
1,김하윤,2,4
2,김하윤,3,3
3,김하윤,4,2
4,김하윤,5,3
...,...,...,...
259,권민서,8,2
260,권민서,9,4
261,권민서,10,3
262,권민서,11,3


In [9]:
# 학생 명단과 학생평가를 통해서 척도별 점수 산출 함수
def calculate_scale_scores_by_student_evaluation(student_list, student_evaluation_df):
    """
    학생 명단과 학생평가 데이터프레임을 받아 학생별로 척도별 점수를 반환하는 함수
    """
    # 정규화
    def normalize(z):
        if z >= 0.84:
            return 5
        elif z >= 0.25:
            return 4
        elif z >= -0.25:
            return 3
        elif z >= -0.84:
            return 2
        else:
            return 1

    # 인기
    popularity_item_no = [2]
    popularity_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(popularity_item_no)].reset_index(drop=True)
    popularity_evaluation_df['응답값'] = popularity_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    popularity_evaluation_mean = popularity_evaluation_df['응답값'].mean()
    print(popularity_evaluation_mean, popularity_evaluation_df['응답값'].std(ddof=0))
    popularity_evaluation_df['인기Z점수'] = (popularity_evaluation_df['응답값'] - popularity_evaluation_mean) / popularity_evaluation_df['응답값'].std(ddof=0)  # 인기 Z점수
    popularity_evaluation_df['인기_정규화'] = popularity_evaluation_df['인기Z점수'].apply(normalize)
    print(popularity_evaluation_df)
    # 선호
    social_preference_item_no = [3]
    social_preference_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(social_preference_item_no)].reset_index(drop=True)
    social_preference_evaluation_df['응답값'] = social_preference_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    social_preference_evaluation_mean = social_preference_evaluation_df['응답값'].mean()
    social_preference_evaluation_df['사회적선호Z점수'] = (social_preference_evaluation_df['응답값'] - social_preference_evaluation_mean) / social_preference_evaluation_df['응답값'].std(ddof=0)  # 사회적 선호 Z점수
    social_preference_evaluation_df['사회적선호_정규화'] = social_preference_evaluation_df['사회적선호Z점수'].apply(normalize)
    # 외현적 공격성
    overt_aggression_item_no = [4]
    overt_aggression_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(overt_aggression_item_no)].reset_index(drop=True)
    overt_aggression_evaluation_df['응답값'] = overt_aggression_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    overt_aggression_evaluation_mean = overt_aggression_evaluation_df['응답값'].mean()
    overt_aggression_evaluation_df['외현적공격성Z점수'] = (overt_aggression_evaluation_df['응답값'] - overt_aggression_evaluation_mean) / overt_aggression_evaluation_df['응답값'].std(ddof=0)  # 외현적 공격성 Z점수
    overt_aggression_evaluation_df['외현적공격성_정규화'] = overt_aggression_evaluation_df['외현적공격성Z점수'].apply(normalize)
    # 관계적 공격성
    relational_aggression_item_no = [5]
    relational_aggression_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(relational_aggression_item_no)].reset_index(drop=True)
    relational_aggression_evaluation_df['응답값'] = relational_aggression_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    relational_aggression_evaluation_mean = relational_aggression_evaluation_df['응답값'].mean()
    relational_aggression_evaluation_df['관계적공격성Z점수'] = (relational_aggression_evaluation_df['응답값'] - relational_aggression_evaluation_mean) / relational_aggression_evaluation_df['응답값'].std(ddof=0)  # 관계적 공격성 Z점수
    relational_aggression_evaluation_df['관계적공격성_정규화'] = relational_aggression_evaluation_df['관계적공격성Z점수'].apply(normalize)
    # 친사회성
    prosocial_item_no = [8]
    prosocial_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(prosocial_item_no)].reset_index(drop=True)
    prosocial_evaluation_df['응답값'] = prosocial_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    prosocial_evaluation_mean = prosocial_evaluation_df['응답값'].mean()
    prosocial_evaluation_df['친사회성Z점수'] = (prosocial_evaluation_df['응답값'] - prosocial_evaluation_mean) / prosocial_evaluation_df['응답값'].std(ddof=0)  # 친사회성 Z점수
    prosocial_evaluation_df['친사회성_정규화'] = prosocial_evaluation_df['친사회성Z점수'].apply(normalize)
    # 가해
    perpetrator_item_no = [6]
    perpetrator_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(perpetrator_item_no)].reset_index(drop=True)
    perpetrator_evaluation_df['응답값'] = perpetrator_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    perpetrator_evaluation_mean = perpetrator_evaluation_df['응답값'].mean()
    perpetrator_evaluation_df['가해Z점수'] = (perpetrator_evaluation_df['응답값'] - perpetrator_evaluation_mean) / perpetrator_evaluation_df['응답값'].std(ddof=0)  # 가해 Z점수
    perpetrator_evaluation_df['가해_정규화'] = perpetrator_evaluation_df['가해Z점수'].apply(normalize)
    # 피해
    victim_item_no = [7]
    victim_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(victim_item_no)].reset_index(drop=True)
    victim_evaluation_df['응답값'] = victim_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    victim_evaluation_mean = victim_evaluation_df['응답값'].mean()
    victim_evaluation_df['피해Z점수'] = (victim_evaluation_df['응답값'] - victim_evaluation_mean) / victim_evaluation_df['응답값'].std(ddof=0)  # 피해 Z점수
    victim_evaluation_df['피해_정규화'] = victim_evaluation_df['피해Z점수'].apply(normalize)
    # 교사 선호
    teacher_preference_item_no = [11]
    teacher_preference_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(teacher_preference_item_no)].reset_index(drop=True)
    teacher_preference_evaluation_df['응답값'] = teacher_preference_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    teacher_preference_evaluation_mean = teacher_preference_evaluation_df['응답값'].mean()
    teacher_preference_evaluation_df['교사선호Z점수'] = (teacher_preference_evaluation_df['응답값'] - teacher_preference_evaluation_mean) / teacher_preference_evaluation_df['응답값'].std(ddof=0)  # 교사 선호 Z점수
    teacher_preference_evaluation_df['교사선호_정규화'] = teacher_preference_evaluation_df['교사선호Z점수'].apply(normalize)
    # 교사 비선호
    teacher_non_preference_item_no = [12]
    teacher_non_preference_evaluation_df = student_evaluation_df[student_evaluation_df['문항번호'].isin(teacher_non_preference_item_no)].reset_index(drop=True)
    teacher_non_preference_evaluation_df['응답값'] = teacher_non_preference_evaluation_df['응답값'].apply(pd.to_numeric, errors='coerce')
    teacher_non_preference_evaluation_mean = teacher_non_preference_evaluation_df['응답값'].mean()
    teacher_non_preference_evaluation_df['교사비선호Z점수'] = (teacher_non_preference_evaluation_df['응답값'] - teacher_non_preference_evaluation_mean) / teacher_non_preference_evaluation_df['응답값'].std(ddof=0)  # 교사 비선호 Z점수
    teacher_non_preference_evaluation_df['교사비선호_정규화'] = teacher_non_preference_evaluation_df['교사비선호Z점수'].apply(normalize)
    
    return pd.DataFrame({
        '학생명': student_list,
        '인기_정규화': popularity_evaluation_df['인기_정규화'],
        '사회적선호_정규화': social_preference_evaluation_df['사회적선호_정규화'],
        '외현적공격성_정규화': overt_aggression_evaluation_df['외현적공격성_정규화'],
        '관계적공격성_정규화': relational_aggression_evaluation_df['관계적공격성_정규화'],
        '친사회성_정규화': prosocial_evaluation_df['친사회성_정규화'],
        '가해_정규화': perpetrator_evaluation_df['가해_정규화'],
        '피해_정규화': victim_evaluation_df['피해_정규화'],
        '교사선호_정규화': teacher_preference_evaluation_df['교사선호_정규화'],
        '교사비선호_정규화': teacher_non_preference_evaluation_df['교사비선호_정규화']
    })

In [10]:
calculate_scale_scores_by_student_evaluation_df = calculate_scale_scores_by_student_evaluation(student_list, student_evaluation_df)
print(calculate_scale_scores_by_student_evaluation_df)

2.909090909090909 1.443137078762504
   학생 이름  문항번호  응답값     인기Z점수  인기_정규화
0    김하윤     2    4  0.755929       4
1    이준서     2    1 -1.322876       1
2    박서연     2    5  1.448864       5
3    최민준     2    1 -1.322876       1
4    정예은     2    4  0.755929       4
5    강도윤     2    4  0.755929       4
6    오서준     2    1 -1.322876       1
7    윤지우     2    5  1.448864       5
8    한예진     2    3  0.062994       3
9    장하람     2    4  0.755929       4
10   조하린     2    3  0.062994       3
11   배수아     2    2 -0.629941       2
12   남윤후     2    1 -1.322876       1
13   서채은     2    3  0.062994       3
14   이아린     2    1 -1.322876       1
15   문지아     2    1 -1.322876       1
16   류하진     2    3  0.062994       3
17   신도현     2    3  0.062994       3
18   홍지우     2    5  1.448864       5
19   유하늘     2    2 -0.629941       2
20   노지아     2    3  0.062994       3
21   권민서     2    5  1.448864       5
    학생명  인기_정규화  사회적선호_정규화  외현적공격성_정규화  관계적공격성_정규화  친사회성_정규화  가해_정규화  피해_정규화  \
0   김하윤   

In [11]:
calculate_scale_scores_by_student_evaluation_df

,학생명,인기_정규화,사회적선호_정규화,외현적공격성_정규화,관계적공격성_정규화,친사회성_정규화,가해_정규화,피해_정규화,교사선호_정규화,교사비선호_정규화
0,김하윤,4,3,3,4,5,1,3,5,1
1,이준서,1,3,2,1,3,1,5,2,5
2,박서연,5,3,2,2,1,2,2,2,3
3,최민준,1,1,2,1,3,1,2,2,5
4,정예은,4,5,2,1,5,2,3,4,1
5,강도윤,4,3,3,4,5,1,3,5,1
6,오서준,1,2,5,2,3,4,5,2,4
7,윤지우,5,5,2,5,5,5,2,5,3
8,한예진,3,3,5,4,3,5,3,2,4
9,장하람,4,5,2,1,5,2,3,4,1


In [12]:
'''
인기 : 조서윤, 정채원 안맞음
'''

'\n인기 : 조서윤, 정채원 안맞음\n'

### 또래지명과 학생평가 일치율 산출 함수

In [13]:
def calculate_euclidean_distance_between_peer_nomination_and_student_evaluation(calculate_scale_scores_by_peer_nomination_df, calculate_scale_scores_by_student_evaluation_df) :
    """
    또래지명 척도별 점수와 학생평가 척도별 점수 간의 유클리드 거리를 계산하는 함수
    """
    euclidean_list = []
    for idx in range(1,len(calculate_scale_scores_by_peer_nomination_df.columns)):
        a_list = calculate_scale_scores_by_peer_nomination_df.iloc[:, idx]
        b_list = calculate_scale_scores_by_student_evaluation_df.iloc[:, idx]
        diff_list = []
        for a, b in zip(a_list, b_list) :
            diff = a - b
            diff_list.append(diff)
        squ_list = [i**2 for i in diff_list]
        sum_value = sum(squ_list)
        root_value = sum_value**0.5
        max_value = (len(calculate_scale_scores_by_peer_nomination_df)*16)**0.5
        euclidean_value = (1-(root_value/max_value))*100
        euclidean_value = max(0, euclidean_value)
        euclidean_list.append(euclidean_value)
    return pd.DataFrame({
        '척도' : list(calculate_scale_scores_by_peer_nomination_df.columns[1:]),
        '일치율' : euclidean_list
    })

In [14]:
euclidean_df = calculate_euclidean_distance_between_peer_nomination_and_student_evaluation(calculate_scale_scores_by_peer_nomination_df, calculate_scale_scores_by_student_evaluation_df)
print(euclidean_df)

           척도        일치율
0      인기_정규화  56.698730
1   사회적선호_정규화  41.856369
2  외현적공격성_정규화  58.371229
3  관계적공격성_정규화  57.028022
4    친사회성_정규화  45.644269
5      가해_정규화  45.644269
6      피해_정규화  49.154870
7    교사선호_정규화  46.966991
8   교사비선호_정규화  45.383569


In [ ]:
'''
인기가 안맞음
'''

'\n인기가 안맞음\n'

## 교사역량 평가

In [15]:
teacher_competency_df = pd.read_excel('5학년_1반_전수(AC000120255019FA1)/AC000120255019FA1_교사_학급평가_응답.xlsx')
print(teacher_competency_df)

                                                 응답값
0  {"questionChoiceList":[{"questionNo":1,"choice...


In [16]:
import json

response_arr = None
for i in range(len(teacher_competency_df['응답값'])):
    json_data = teacher_competency_df['응답값'][i]
    json_data = json.loads(json_data)
    response_json_data = json_data['questionChoiceList']
    response_data = []

    for item in response_json_data:
        print(item)
        response_data.append(item['choiceScore'])
    if response_arr is None:
        response_arr = np.array(response_data)
        print(response_arr)
    else:
        response_arr = np.vstack([response_arr, response_data])

{'questionNo': 1, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 2, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 3, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 4, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 5, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 6, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 7, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 8, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 9, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 10, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 11, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 12, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 13, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 14, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 15, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 16, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 17, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 18, 'choiceNo': '3', 'choiceScore': '3'}
{

In [17]:
def calculate_teacher_competency_scores(teacher_competency_df):
    """
    교사 평가 json으로 교사별 역량 점수를 계산하는 함수
    """
    response_arr = None
    for i in range(len(teacher_competency_df['응답값'])):
        json_data = teacher_competency_df['응답값'][i]
        json_data = json.loads(json_data)
        response_json_data = json_data['questionChoiceList']
        response_data = []

        for item in response_json_data:
            print(item)
            response_data.append(item['choiceScore'])
        if response_arr is None:
            response_arr = np.array(response_data)
            print(response_arr)
        else:
            response_arr = np.vstack([response_arr, response_data])
    teacher_competency_df = pd.DataFrame([response_arr])
    teacher_competency_df.columns = [i for i in range(1, 24)]
    teacher_competency_df[[i for i in range(1,24)]] = teacher_competency_df[[i for i in range(1,24)]].apply(pd.to_numeric, errors='coerce')

    # 평등한 지위관리
    equalstatus_management_item_no = [1,2,3,4,5,6]
    equalstatus_management_df = teacher_competency_df[equalstatus_management_item_no]
    equalstatus_management_value = equalstatus_management_df.mean(axis=1).iloc[0]
    # 소외 학생 관리
    marginalized_student_management_item_no = [7,8,9,10]
    marginalized_student_management_df = teacher_competency_df[marginalized_student_management_item_no]
    marginalized_student_management_value = marginalized_student_management_df.mean(axis=1).iloc[0]
    # 문제행동 관리
    problem_behavior_management_item_no = [11,12,13]
    problem_behavior_management_df = teacher_competency_df[problem_behavior_management_item_no]
    problem_behavior_management_value = problem_behavior_management_df.mean(axis=1).iloc[0]
    # 공격성 관리
    aggression_management_item_no = [14,15,16,17,18]
    aggression_management_df = teacher_competency_df[aggression_management_item_no]
    aggression_management_value = aggression_management_df.mean(axis=1).iloc[0]
    # 긍정적 분위기 조성
    positive_atmosphere_item_no = [19, 20, 21, 22, 23]
    positive_atmosphere_df = teacher_competency_df[positive_atmosphere_item_no]
    positive_atmosphere_value = positive_atmosphere_df.mean(axis=1).iloc[0]

    return pd.DataFrame([{
        '평등한지위관리': equalstatus_management_value,
        '소외학생관리': marginalized_student_management_value,
        '문제행동관리': problem_behavior_management_value,
        '공격성관리': aggression_management_value,
        '긍정적분위기조성': positive_atmosphere_value
    }])

In [18]:
df = calculate_teacher_competency_scores(teacher_competency_df)

{'questionNo': 1, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 2, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 3, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 4, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 5, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 6, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 7, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 8, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 9, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 10, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 11, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 12, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 13, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 14, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 15, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 16, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 17, 'choiceNo': '3', 'choiceScore': '3'}
{'questionNo': 18, 'choiceNo': '3', 'choiceScore': '3'}
{

In [19]:
df

,평등한지위관리,소외학생관리,문제행동관리,공격성관리,긍정적분위기조성
0,3.0,3.0,3.0,3.0,3.0


In [ ]:
'''
이건 왜 첫째자리 반올림이지?
'''

'\n이건 왜 첫째자리 반올림이지?\n'